<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.877 · Análisis de sentimientos y textos</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencias de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicaciones</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>


# PAC 1: Procesamiento y análisis de información textual

En esta práctica revisaremos y aplicaremos los conocimientos aprendidos en el módulo 1. Concretamente trataremos 3 temas.

<ul>
<li>1. Obtención de datos a partir de información textual
<li>2. Detección de temas
<li>3. Clasificación de textos
</ul>

El propósito de la práctica es descubrir rasgos característicos de las noticias falsas sobre el Covid-19 usando las herramientas explicadas en el módulo 1. Además veremos si es posible clasificar automáticamente noticias falsas con métodos de machine learning. Utilizaremos el dataset <i>corona_fake.csv</i>. Este dataset contiene noticias en inglés sobre el covid-19 etiquetadas según si son noticias falsas (<i>fake</i>) o no. El dataset se organiza en cuatro columnas:

<b>title</b>: títular de la noticia<br>
<b>text</b>: cuerpo de la noticia<br>
<b>source</b>: fuente de la noticia<br>
<b>label</b>: etiqueta <i>Fake</i> si la noticia es falsa. Etiqueta <i>TRUE</i> si es verdadera

In [1]:
import nltk
#nltk.download('all')
import pandas as pd

In [2]:
df= pd.read_csv("corona_fake.csv")

df.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


## 1. Obtención de datos a partir de información textual (5 puntos)

Primero, cargamos las librerías necesarias.


Crearemos dos dataframes. El primero (df_fake) contendrá las noticias clasificadas como <i>Fake</i> y el segundo dataframe (df_true) contendrá las noticias clasificadas como <i>TRUE</i>

In [3]:
df_fake = df.loc[df['label'] == 'Fake']

df_fake.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [4]:
df_true = df.loc[df['label'] == 'TRUE']

df_true.head()

,title,text,source,label
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE
16,Being able to hold your breath for 10 seconds ...,The most common symptoms of COVID-19 are dry c...,https://www.who.int/emergencies/diseases/novel...,TRUE
17,Drinking alcohol does not protect you against ...,Frequent or excessive alcohol consumption can ...,https://www.who.int/emergencies/diseases/novel...,TRUE
18,COVID-19 virus can be transmitted in areas wit...,"From the evidence so far, the COVID-19 virus c...",https://www.who.int/emergencies/diseases/novel...,TRUE


## 1.1 Encontrar colocaciones (2 puntos)

Recordad que las colocaciones son términos multipalabra, es decir, secuencias de palabras que tienen un significado en conjunto significativamente diferente del significado derivado de los significados de las palabras individuales  (e.g. New York tiene un significado distinto del que se puede derivar de New y de York).

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong>  Computa los mejores bigramas y trigramas de los titulares de las noticias falsas que no están en los titulares de las noticias verdaderas (1 punto)
</div>

In [5]:
#Importar la lista de stopwords en inglés de la libreria NLTK.
stopwords = nltk.corpus.stopwords.words('english')
#Añadir stopwords
stopwords = stopwords + ['unknown', 've', 'hadn', 'll', 'didn', 'isn', 'doesn', 'hasn' ]

Para este apartado hay que cargar las siguientes librerías:

In [6]:
from nltk import pos_tag, word_tokenize
from nltk.collocations import *
import re

A partir del comando help(nltk.collocations.BigramAssocMeasures) explora la clase BigramAssocMeasures del módulo nltk.metrics.association y revisa las definiciones de las métricas de Likelihood Ratio (likelihood_ratio) y de Pointwise Mutual Information (pmi) en las secciones que se indican del capítulo 5 del libro Foundations of Statistical Natural Language Processing (Manning & Schutze).

In [7]:
help(nltk.collocations.BigramAssocMeasures)

Help on class BigramAssocMeasures in module nltk.metrics.association:

class BigramAssocMeasures(NgramAssocMeasures)
 |  A collection of bigram association measures. Each association measure
 |  is provided as a function with three arguments::
 |  
 |      bigram_score_fn(n_ii, (n_ix, n_xi), n_xx)
 |  
 |  The arguments constitute the marginals of a contingency table, counting
 |  the occurrences of particular events in a corpus. The letter i in the
 |  suffix refers to the appearance of the word in question, while x indicates
 |  the appearance of any word. Thus, for example:
 |  
 |  - n_ii counts ``(w1, w2)``, i.e. the bigram being scored
 |  - n_ix counts ``(w1, *)``
 |  - n_xi counts ``(*, w2)``
 |  - n_xx counts ``(*, *)``, i.e. any bigram
 |  
 |  This may be shown with respect to a contingency table::
 |  
 |              w1    ~w1
 |           ------ ------
 |       w2 | n_ii | n_oi | = n_xi
 |           ------ ------
 |      ~w2 | n_io | n_oo |
 |           ------ ------
 |  

Primer paso: Computa los tokens de los titulares de las noticias falsas. Etiqueta estos tokens por su PoS. Ten en cuenta que hay noticias sin titular y que puede haber titulares con palabras que tengan caracteres especiales al principio. Si una noticia no tiene titular (NaN en la columna 'title') sustituimos NaN por 'empty'

In [8]:
#Sustituimos 'Nan' por 'empty'
df_no_na = df.fillna('empty')

#Creamos un dataframe que contiene las noticias falsas
df_fake = df_no_na.loc[df['label'] == 'Fake']

#Creamos un dataframe que contiene las noticias verdaderas
df_true = df_no_na.loc[df['label'] == 'TRUE']

#Quitamos los titulares vacíos 

titulares_fake_noempty = [fh for fh in df_fake['title'].to_list() if fh != 'empty']
titulares_true_noempty = [th for th in df_true['title'].to_list() if th != 'empty']

#Creamos un texto en minúscula con todos los titulares falsos

titulares_fake = " ".join(titulares_fake_noempty).lower()

#Creamos un texto en minúscula con todos los titulares verdaderos

titulares_true = " ".join(titulares_true_noempty).lower()

In [9]:
#############################################
# SOLUCIÓN                                   #
#############################################
from nltk import word_tokenize
alpha_tokens = [w for w in word_tokenize(titulares_fake) if re.match("^[a-z]+.*", w)]
tagged_tokens = nltk.pos_tag(alpha_tokens)


Segundo paso: Computa los 1000 mejores bigramas y los 1000 mejores trigramas a partir de los tokens etiquetados (e.g. [(Basic, JJ), ...]) de los titulares falsos. Utiliza las métricas PMI y la Likehood Ratio. Tienes que comentar las similitudes y diferencias que encuentras en los resultados según la métrica utilizada.

<b>Atención</b>: De los 1000 bigramas y trigramas, elige los que no empiezan ni terminan con una stopword.

Recordemos la clasificación de etiquetas PoS.

<b>Etiquetas PoS</b>

<ul>
<li>DT: Determinante</li>
<li>JJ: Adjetivo</li>
<li>NN: Nombre en singular</li>
<li>NNS: Nombre en plural</li>
<li>VBD: Verbo en pasado</li>
<li>VBG: Verbo en gerundio</li>
<li>MD: Verbo modal</li>
<li>IN: Preposición o conjunción subordinada</li>
<li>PRP: Pronombre</li>
<li>RB: Adverbio</li>
<li>RP: Partícula</li>    
<li>CC: Conjunción coordinada</li>
<li>CD: Numeral</li>
</ul>

In [10]:

from nltk.util import ngrams
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

def get_coll_candidates(tokens):
    bigramcandidates = BigramCollocationFinder.from_words(tokens)
    trigramcandidates = TrigramCollocationFinder.from_words(tokens)
    return bigramcandidates , trigramcandidates

def get_n_best_candidates(bigram_candidates, trigram_candidates, n_best_collocations, metrica='pmi'):
    if metrica == 'pmi':
        nbest_bigram_candidates = bigram_candidates.nbest(bigram_measures.pmi,n_best_collocations)
        nbest_trigram_candidates = trigram_candidates.nbest(trigram_measures.pmi,n_best_collocations)
    
    else:
        nbest_bigram_candidates = bigram_candidates.nbest(bigram_measures.likelihood_ratio,n_best_collocations)
        nbest_trigram_candidates = trigram_candidates.nbest(trigram_measures.likelihood_ratio,n_best_collocations)
      
    return nbest_bigram_candidates, nbest_trigram_candidates

def good_stw_candidate(candidate):
    test = True
    if type(candidate) == str:
        if candidate in stopwords:
            test = False
    else:
        if len(candidate)==1:
            if candidate[0] in stopwords:
                test = False
        else:
            if len(candidate)==2:
                if candidate[0][0] in stopwords or candidate[-1][0] in stopwords:
                    test = False
            else:
                if candidate[0][0] in stopwords or candidate[1][0] in stopwords or candidate[-1][0] in stopwords:
                    test = False
    return test


In [11]:
#############################################
# SOLUCIÓN                                  #
#############################################

bigram_coll_candidates, trigram_coll_candidates = get_coll_candidates(tagged_tokens)

nbest_bigram_candidates, nbest_trigram_candidates = get_n_best_candidates(bigram_coll_candidates, 
                                                                          trigram_coll_candidates, 
                                                                          1000)
nbest_bigram_candidates_lr, nbest_trigram_candidates_lr = get_n_best_candidates(bigram_coll_candidates, 
                                                                          trigram_coll_candidates, 
                                                                          1000,
                                                                          'lr')
nbc_pmi=[] #bigramas con metrica pmi sin stopword
nbc_lr=[]  #bigramas con metrica lr sin stopword
ntc_pmi=[] #trigramas con metrica pmi sin stopword
ntc_lr=[]  #trigramas con metrica lr sin stopword

for i in range(0,1000):
    if good_stw_candidate(nbest_bigram_candidates[i]):
        nbc_pmi.append(nbest_bigram_candidates[i])
    if good_stw_candidate(nbest_bigram_candidates_lr[i]):
        nbc_lr.append(nbest_bigram_candidates_lr[i]) 
    if good_stw_candidate(nbest_trigram_candidates[i]):
        ntc_pmi.append(nbest_trigram_candidates[i])
    if good_stw_candidate(nbest_trigram_candidates_lr[i]):
        ntc_lr.append(nbest_trigram_candidates_lr[i])  
        
        
        
print('Bigramas con métrica PMI  : ',len(nbc_pmi))
print('Bigramas con métrica LR   : ',len(nbc_lr))
print('Trigramas con métrica PMI : ',len(ntc_pmi))
print('Trigramas con métrica LR  : ',len(ntc_lr))

Bigramas con métrica PMI  :  761
Bigramas con métrica LR   :  690
Trigramas con métrica PMI :  527
Trigramas con métrica LR  :  430


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong>  Escribe un comentario sobre los resultados obtenidos, contestando las siguientes preguntas: ¿Con las métricas utilizadas, es posible encontrar características distintivas en los ngramas de los titulares de noticias falsas? ¿Qué métrica te parece más adecuada para realizar este análisis? ¿Sería más eficiente detectar solamente ngramas que cumplen el patrón sintáctico de un sintagma nominal (e.g: adjetivo + nombre en singular/plural y nombre + nombre)? (1 punto)
</div>

In [12]:
#############################################
# SOLUCIÓN                                  #
#############################################


### 1.2 Vectorizar palabras y términos (3 puntos)

Exploraremos la vectorización de palabras y términos con el método Word2Vec.

Recordemos que el paquete gensim implementa un método para entrenar modelos Word2Vec.

In [13]:
import gensim

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Obtén los términos relacionados con 'coronavirus' en las noticias falsas y los términos relacionados con 'coronavirus' en las noticias verdaderas. Utiliza el cálculo de similitud semántica de un modelo word2vec (2 puntos)
</div>

Primer paso: Entrena un modelo de detección de phrases en una oración. Para el entrenamiento utiliza todos los titulares y los cuerpos (no vacíos) de las noticias falsas y verdaderas. Utiliza el módulo Phraser de Gensim

In [14]:
#############################################
# SOLUCIÓN                                  #
#############################################
from gensim.models.phrases import Phraser
from gensim.models import Phrases

ss1 = [fh for fh in df_fake['title'].to_list() if fh != 'empty'] 
ss2 = [th for th in df_true['title'].to_list() if th != 'empty']
ss3 = [ft for ft in df_fake['text'].to_list() if ft != 'empty']
ss4 = [tt for tt in df_true['text'].to_list() if tt != 'empty']
sentence_stream = ss1 + ss2 + ss3 + ss4 
sentence_stream = " ".join(sentence_stream).lower()

text_stream = [w for w in word_tokenize(sentence_stream) if re.match("^[a-z]+.*", w)]

phrases = Phrases(text_stream, min_count=1, threshold=2)

phraser = Phraser(phrases)


Segundo paso: Transforma cada frase de las noticias fake en una lista de phrases lematizadas

<b>Atención</b>: Las phrases no deben ser stopwords. Tampoco deben empezar ni terminar con una stopword.

In [40]:
#############################################
# SOLUCIÓN                                  #
#############################################
from nltk.stem.wordnet import WordNetLemmatizer
no_pos_in = ['DT', 'IN', 'PRP', 'CC', 'CD','MD', 'VBG', 'VBD', 'RP', 'RB']
def get_wn_pos(pos):
    if re.match(r'^N',pos):
        wn_pos = 'n'
    elif re.match(r'^V',pos):
        wn_pos = 'v'
    else:
        wn_pos = 'n' #En inglés, los lemas de términos que no son verbos ni nombres se obtienen como si fueran
                        #nombres
    return wn_pos

#La función wnlemmatize lematiza el término con una etiqueta PoS según el lematizador de Wordnet
def wnlemmatize(t,postag):
    lemma = ""
    #Definición del lematizador
    lem = WordNetLemmatizer()
    #Si el candidato es monopalabra, se obtiene el lema con el lematizador de WordNet según su PoS
    if ' ' not in t:
        #lemma = lem.lemmatize(t,get_wn_pos(postag[0][1]))
        lemma = lem.lemmatize(t,get_wn_pos(postag[1]))
    #Si el candidato es multipalabra, obtenemos su lema como si fuera un nombre, aplicando el lematizador de WordNet
    else:
        lemma = lem.lemmatize(t,'n')
    return lemma

def transform_sentence(ss):
    ss=ss.lower()
    text_stream = [w for w in word_tokenize(ss) if re.match("^[a-z]+.*", w) 
                                            and good_stw_candidate(w)
                                            and nltk.pos_tag([w])[0][1] not in no_pos_in 
              ]
    text_phrases=phraser[text_stream]

    tagged_phrases = nltk.pos_tag(text_phrases)
    lemmas=[]
    for tm in range(0,len(text_phrases)):
        lemmas.append(wnlemmatize(text_phrases[tm],tagged_phrases[tm]))
    return lemmas


ss=ss1+ss3 #titulares y texto de las noticias falsas
#ss=" ".join(ss).lower()
ss=ss.lower()
lemmas =  [transform_sentence(i) for i in ss]
print([6])

AttributeError: 'list' object has no attribute 'lower'

In [38]:
print(transform_sentence(ss[4]))
print(ss[4].lower())

[]
covid and the terror of uncertainty


Tercer paso: Crear un modelo word2vec de los titulares y los cuerpos de noticias falsas

In [23]:
#############################################
# SOLUCIÓN                                  #
#############################################
def transform_sentence(sentence):
    #Obtenemos los phrases según el modelo de detección de phrases que ha aprendido
    sentence_phrases = phrases[word_tokenize(sentence.lower())]
    #Quitamos los signos de puntuación de los phrases
    phrases_stripped = [st.strip('".,;:-():!?-‘’ ') for st in sentence_phrases if re.match("^[a-z]+.*", st)]
    #Hacemos etiquetaje de PoS de los phrases y lo guardamos en un diccionario (postag) 
    for ps in phrases_stripped:
        postag[ps] = nltk.pos_tag(word_tokenize(ps))
    #Lematizamos los phrases con el lematizador de Wordnet
    phrases_lemmatized = [wnlemmatize(ps,postag[ps]) for ps in phrases_stripped]
    #Unificamos los phrases
    phrases_lemmatized_and_unified = [unify(pl) for pl in phrases_lemmatized ]
    return phrases_lemmatized_and_unified




sentence_stream=ss1+ss3
postag={}
print(sentence_stream[0])
#transformed_sentences = [transform_sentence(ss) for ss in sentence_stream ]
#print(transformed_sentences[:50])

Due to the recent outbreak for the Coronavirus (COVID-19) the World Health Organization is giving away vaccine kits. Just pay $4.95 for shipping


Cuarto paso: Seleccionar el vocabulario del modelo word2vec sobre el cual se verán los términos parecidos

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



Quinto paso: Presentar los términos fake más cercanos semánticamente a 'coronavirus'

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



Sexto paso: Realiza los pasos anteriores pero con las frases de las noticias verdaderas

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################




<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Comenta las diferencias que existen en los términos relacionados con 'coronavirus' en las noticias falsas y en las noticias verdaderas. ¿Crees que estas diferencias son representativas de los contenidos de las noticias falsas? (1 punto)
</div>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



## 2. Detección de temas. (4 puntos)

En estos apartados exploraremos los temas tratados en las noticias falsas.

### 2.1 Exploración de los temas con WordNet (2 puntos)


En este apartado accederemos a Wordnet a través de la librería nltk.

In [ ]:
from nltk.corpus import wordnet as wn

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Comprueba si las noticias falsas contienen términos alejados semánticamente del sentido del término 'disease' en Wordnet. Compruébalo calculando la similitud de Wu and Palmer entre el sentido de wordnet 'disease.n.01' y los términos relacionados con 'coronavirus' en el modelo word2vec de las noticias falsas. (1 punto)
</div>

Primer paso: Calcula la distancia Wu and Palmer entre el sentido 'disease.n.01' y el primer sentido de los sustantivos más relacionados con 'coronavirus' en el modelo word2vec de las noticias falsas.

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Comenta los resultados que te han llamado más la atención. Fíjate en los términos que no están en Wordnet y los que tienen una distancia muy alejada al sentido de disease.n.01. ¿Crees que Wordnet es un buen recurso para analizar los temas que se tratan en las noticias falsas sobre el coronavirus? (1 punto)
</div>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



### 2.2 LDA (2 puntos)

Recordemos que en el notebook del módulo 1 hemos visto la aplicación del método LDA para extraer temas de documentos.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Extrae temas a partir de los phrases de los titulares y los cuerpos de noticias falsas. Lo haremos con el método LDA. Experimenta con el parámetro num_topics hasta encontrar un conjunto de temas informativo, y asigna nombres a los temas encontrados. La construcción del modelo LDA puede tardar más de 10 minutos en algunos casos (2 puntos)
</div>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



## 3. Clasificación (1 punto)


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Crea un clasificador automático de noticias falsas y no falsas a partir de los titulares. (0.5 puntos)
</div>

Primer paso: Unimos el dataframe con las noticias falsas y el dataframe con las noticias verdaderas

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



Segundo paso: Realizamos dos listas. Una con los titulares y otra con las etiquetas correspondientes

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



Tercer paso: Vectorizamos los titulares con un vectorizador tf.idf

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



Cuarto paso:Preparamos el corpus de entrenamiento y de evaluación

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



Quinto paso: Entrenar el clasificador con Logistic Regression

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



Sexto paso: Utilizar el modelo entrenado para predecir la categoría Fake o True de los titulares del conjunto de test y mostrar las palabras más informativas para cada categoría. 

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################



<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> A partir de las palabras más informativas, ¿qué contenidos crees que son típicos de las noticias falsas sobre el coronavirus? ¿Crees que hay que considerar elementos formales (e.g: ausencia de titular, uso de mayúsculas) como distintivos de las noticias falsas? (0.5 puntos)
</div>

## Solución



